# This is the file whereby the data to transfermarkt is being scraped

In [ ]:
!pip install unidecode
!pip install shap
!pip3 install catboost
!pip insert
from keras.models import *
from unidecode import unidecode
import time
import numpy as np
import pandas as pd
import os
import sklearn
from sklearn import datasets
from urllib.error import URLError
from tensorflow.keras.layers import Dense
import pickle
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
from keras.applications.xception import Xception 
import tensorflow as tf
import shap
from sklearn.inspection import permutation_importance
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.linear_model import Perceptron
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.neural_network import MLPClassifier
from sklearn import preprocessing
from sklearn.linear_model import LinearRegression
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from shap.plots import *
import keras

In [ ]:
!pip install wandb -qqq
import wandb
# Log in to your W&B account
wandb.login()

# Mount the drive holding the dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)


In [ ]:
dataset_path = '/content/drive/MyDrive/FYPDataset'
dataset_filename = '/footballData.csv'
dataframe_fifa = pd.read_csv(dataset_path + dataset_filename)

In [ ]:
dataframe_fifa

In [ ]:
dataframe_fifa['Index Of Player Found'] = dataframe_fifa.index

In [ ]:
dataframe_fifa.columns

In [ ]:
dataframe_fifa.loc[dataframe_fifa['short_name'] == 'Pedri']

In [ ]:
shortNamesListFifa = dataframe_fifa['short_name'].tolist()
len(shortNamesListFifa)

In [ ]:
longNamesListFifa = dataframe_fifa['long_name'].tolist()
len(longNamesListFifa)

In [ ]:
def removeAccentsFromString(list_of_players):
  names_cleaned = []
  for i in range(len(list_of_players)):
    nameWithoutAccents = unidecode(list_of_players[i])
    names_cleaned.append(nameWithoutAccents)
  return names_cleaned

In [ ]:
shortNamesCleaned = removeAccentsFromString(shortNamesListFifa)
longNamesCleaned = removeAccentsFromString(longNamesListFifa)

In [ ]:
len(shortNamesCleaned)

In [ ]:
len(longNamesCleaned)

In [ ]:
dataframe_fifa.insert(0, "shortNamesCleaned", shortNamesCleaned)  
dataframe_fifa.insert(1, "longNamesCleaned", longNamesCleaned)

In [ ]:
dataframe_transfermarkt = pd.read_csv("/content/drive/MyDrive/footballTeamData/joinedDataframe.csv")

In [ ]:
dataframe_transfermarkt

In [ ]:
columnsTransfermarkt = dataframe_transfermarkt.columns.tolist()

In [ ]:
dataframe_transfermarkt.drop(columns = columnsTransfermarkt[0], axis = 1, inplace = True) 

In [ ]:
dataframe_transfermarkt

In [ ]:
name_list = dataframe_transfermarkt["Players"].tolist()

In [ ]:
print(name_list)

In [ ]:
cleanedTransfermarkt = removeAccentsFromString(name_list)

In [ ]:
print(cleanedTransfermarkt[0])

In [ ]:
def indexOfPlayersInBothDataframesTransfermarktValue(playersName):
  listOfTheIndexOfPlayersInBoth = []
  playersThatCannotBeFound = []
  moreThanOneIndexForPlayer = []
  playerNameFound = []
  for i in range(len(playersName)):
    playerIndex = dataframe_fifa.loc[(dataframe_fifa['shortNamesCleaned'] == playersName[i]) | (dataframe_fifa['longNamesCleaned'] == playersName[i])].index
    if(playerIndex.size > 1):
      moreThanOneIndexForPlayer.append(playerIndex)
      continue
    if(playerIndex.size == 0):
      playersThatCannotBeFound.append(playersName[i])
    else:
      listOfTheIndexOfPlayersInBoth.append(playerIndex[0])
      playerNameFound.append(playersName[i])

  return listOfTheIndexOfPlayersInBoth, playersThatCannotBeFound, playerNameFound

  


In [ ]:
playersFound = indexOfPlayersInBothDataframesTransfermarktValue(cleanedTransfermarkt)[0]

In [ ]:
playersNotFoundList = indexOfPlayersInBothDataframesTransfermarktValue(cleanedTransfermarkt)[1]

In [ ]:
playerNameFound = indexOfPlayersInBothDataframesTransfermarktValue(cleanedTransfermarkt)[2]

In [ ]:
len(playersFound) #3461 is the total number of players in the dataset

In [ ]:
playersFoundDF = pd.DataFrame()

In [ ]:
playersFoundDF.insert(0, 'Player Names Found', playerNameFound)

In [ ]:
playersFoundDF.insert(1, 'Index Of Player Found', playersFound)

In [ ]:
print(playersFound) #players found is here 

In [ ]:
print(playersNotFoundList)

In [ ]:
dataframe_fifa.loc[(dataframe_fifa['shortNamesCleaned'] == 'Alvaro Negredo')]

In [ ]:
def checkForPlayersUsingContains(list_of_players): 
  indexOfPlayers = []
  moreThanOneIndexForPlayer = []
  playersThatCannotBeFound = []
  playerNameFound = []
  for i in range(len(list_of_players)):
    playerIndex = dataframe_fifa.loc[dataframe_fifa['longNamesCleaned'].str.contains(list_of_players[i], case=False) | dataframe_fifa['shortNamesCleaned'].str.contains(list_of_players[i], case=False)].index
    if(playerIndex.size > 1):
      moreThanOneIndexForPlayer.append(playerIndex)
      continue
    if(playerIndex.size == 0):
      playersThatCannotBeFound.append(list_of_players[i])
    else:
      indexOfPlayers.append(playerIndex[0])
      playerNameFound.append(list_of_players[i])

  return indexOfPlayers, playersThatCannotBeFound, moreThanOneIndexForPlayer, playerNameFound

In [ ]:
playersRemaining = checkForPlayersUsingContains(playersNotFoundList)

In [ ]:
len(playersNotFoundList)

In [ ]:
playersFoundDFSecondTime = pd.DataFrame()

In [ ]:
playersFoundDFSecondTime.insert(0, 'Player Names Found', playersRemaining[3])
playersFoundDFSecondTime.insert(1, 'Index Of Player Found', playersRemaining[0])

In [ ]:
print(playersRemaining[0]) #all the players index here 

In [ ]:
print(len(playersRemaining[0]))

In [ ]:
print(playersRemaining[1])

In [ ]:
print(len(playersRemaining[1]))

In [ ]:
listOfPlayersRemaining = playersRemaining[1]

In [ ]:
playerNameFoundv2 = playersRemaining[3]

In [ ]:
def splitPlayerNames(list_of_players):
  listOfNames = []
  for i in range(len(list_of_players)):
    splitNames = list_of_players[i].split(" ")
    for j in range(len(splitNames)):
      listOfNames.append(splitNames[j])
  return listOfNames

In [ ]:
def checkForPlayerNamesWhichAreSplitUsingContains(list_of_players): 
  indexOfPlayers = []
  playersThatCannotBeFound = []
  moreThanOneIndexForPlayer = []
  playerNameFound = []
  j = 1
  for i in range(0,len(list_of_players),2):
    playerIndex = dataframe_fifa.loc[dataframe_fifa['longNamesCleaned'].str.contains(list_of_players[i], case=False) & dataframe_fifa['longNamesCleaned'].str.contains(list_of_players[j], case=False)].index
    if(playerIndex.size > 1):
      moreThanOneIndexForPlayer.append(playerIndex)
      j +=2
      continue
    if(playerIndex.size == 0):
      playersThatCannotBeFound.append(list_of_players[i])
      playersThatCannotBeFound.append(list_of_players[j])
    else:
      indexOfPlayers.append(playerIndex[0])
      playerNameFound.append(list_of_players[i] + " " + list_of_players[j])
     

    j +=2

  return indexOfPlayers, playersThatCannotBeFound, moreThanOneIndexForPlayer, playerNameFound

In [ ]:
splitPlayerNames = splitPlayerNames(listOfPlayersRemaining)

In [ ]:
print(splitPlayerNames)

In [ ]:
playersFoundUsingAndCondition = checkForPlayerNamesWhichAreSplitUsingContains(splitPlayerNames)

In [ ]:
playerNameFoundv3 = playersFoundUsingAndCondition[3]

In [ ]:
playersFoundDFThirdTime = pd.DataFrame()
playersFoundDFThirdTime.insert(0, 'Player Names Found', playerNameFoundv3)
playersFoundDFThirdTime.insert(1, 'Index Of Player Found', playersFoundUsingAndCondition[0])

In [ ]:
print(len(playersFoundUsingAndCondition[0]))

In [ ]:
print(len(playersFoundUsingAndCondition[1]))

In [ ]:
playersFoundUsingAndCondition[1][2]

In [ ]:
playersFoundUsingAndCondition[1][3]

In [ ]:
#Emiliano Velazquez
dataframe_fifa.loc[dataframe_fifa['longNamesCleaned'].str.contains(playersFoundUsingAndCondition[1][2], case=False) & dataframe_fifa['longNamesCleaned'].str.contains(playersFoundUsingAndCondition[1][3], case=False) ]

In [ ]:
playersFoundDF

In [ ]:
playersFoundDFSecondTime

In [ ]:
playersFoundDFThirdTime

In [ ]:
nameDf = pd.concat([playersFoundDF , playersFoundDFSecondTime , playersFoundDFThirdTime])

In [ ]:
nameDf

# using all the methods to get the player indexes of the players found in transfermarkt

In [ ]:
playersFound #3461 is the total number of players in the dataset

In [ ]:
len(playersFound)

In [ ]:
len(playersRemaining[0])

In [ ]:
len(playersFoundUsingAndCondition[0])

In [ ]:
arrayOfAllIndexes =np.concatenate((playersFound, playersRemaining[0], playersFoundUsingAndCondition[0]), axis=None)

In [ ]:
arrayOfAllIndexes = arrayOfAllIndexes.tolist()

In [ ]:
len(arrayOfAllIndexes)

In [ ]:
arrayOfAllIndexes

# All the player names and the indexes of the players that are both in the transfermarkt and the fifa dataset  



In [ ]:
playerNamesFoundInBothDataset = np.concatenate((playerNameFound, playerNameFoundv2, playerNameFoundv3), axis=None)

In [ ]:
playerNamesFoundInBothDataset = playerNamesFoundInBothDataset.tolist()

In [ ]:
playerNamesFoundInBothDataset

In [ ]:
'Thomas Partey' in playerNamesFoundInBothDataset

In [ ]:
from pandas import DataFrame
dfPlayersNamesFound = DataFrame(playerNamesFoundInBothDataset,columns=['Player Names'])

In [ ]:
dfPlayersNamesFound

In [ ]:
dataframe_transfermarkt

In [ ]:
playersDFList = dataframe_transfermarkt['Players'].tolist()

In [ ]:
namesRemovedWithAccentsForFinalDF = removeAccentsFromString(playersDFList)

In [ ]:
dataframe_transfermarkt.insert(1, "Player Names", namesRemovedWithAccentsForFinalDF)

In [ ]:
dataframe_transfermarkt

In [ ]:
dataframe_transfermarkt.drop(['Players'], axis = 1, inplace = True) 

In [ ]:
dataframe_transfermarkt

In [ ]:
nameDf

In [ ]:
sortedNameDF = nameDf.sort_values(by='Player Names Found', ascending=True)

In [ ]:
sortedNameDF.drop_duplicates(subset=['Index Of Player Found'])

In [ ]:
sortedNameDF.rename(columns={'Player Names Found':'Player Names'}, inplace=True)

In [ ]:
Both_DFs = pd.merge(sortedNameDF,dataframe_transfermarkt, how='left',left_on=['Player Names'],right_on=['Player Names'])

In [ ]:
Both_DFs.drop_duplicates(subset=['Index Of Player Found'])


In [ ]:
dataframe_transfermarkt[dataframe_transfermarkt['Player Names'] == 'Lionel Messi'].values

In [ ]:
bothSortedDF = Both_DFs.sort_values(by='Index Of Player Found', ascending= True)

In [ ]:
droppedDupDF = bothSortedDF.drop_duplicates(subset=['Index Of Player Found'])

In [ ]:
ValuesList = droppedDupDF['Values'].values.tolist()

In [ ]:
droppedDupDF = droppedDupDF.append({'Player Names':'Lionel Messi', 'Index Of Player Found':0, 'Values':79016681.29968}, ignore_index=True)

In [ ]:
ValuesList.insert(0,79016681.29968)

In [ ]:
droppedDupDF = droppedDupDF.sort_values(by='Index Of Player Found', ascending= True)

In [ ]:
FifaBoth_DFs = pd.merge(dataframe_fifa,droppedDupDF, how='left',left_on=['Index Of Player Found'],right_on=['Index Of Player Found'])

In [ ]:
# FifaBoth_DFs.dropna('Value')
FifaBoth_DFs = FifaBoth_DFs[FifaBoth_DFs['Values'].notna()]
FifaBoth_DFs

In [ ]:
FifaBoth_DFs.drop(columns = ['Index Of Player Found', 'Player Names'], inplace=True)

In [ ]:
FifaBoth_DFs

In [ ]:
FifaBoth_DFs.drop(['real_face','player_tags','sofifa_id','player_url','long_name','dob','league_name','international_reputation','player_tags','loaned_from','team_jersey_number','joined','nation_position','international_reputation','nation_jersey_number','nationality','body_type','player_traits','lwb','rcm','cm','rdm','lb','rwb', 'rw','lm','lw', 'lcm', 'lcb','cb', 'rcb', 'rm','st','cf','lf','rf','lam','cam', 'ram','ls','rs', 'rb', 'ldm', 'cdm','team_position', 'shortNamesCleaned',
 'longNamesCleaned', 'gk_diving','gk_handling','gk_reflexes','gk_positioning','gk_kicking'], axis=1, inplace=True)

In [ ]:
FifaBoth_DFs

In [ ]:
listOfColumns = FifaBoth_DFs.columns.tolist()
listOfColumns

In [ ]:
listOfColumns[18]

In [ ]:
from sklearn.preprocessing import MinMaxScaler

FifaBoth_DFs['attacking_work_rate'] = FifaBoth_DFs['work_rate'].map(lambda x: x.split('/')[0])
FifaBoth_DFs['defensive_work_rate'] = FifaBoth_DFs['work_rate'].map(lambda x: x.split('/')[1])
FifaBoth_DFs.drop(columns = ['work_rate'], inplace=True) 
cleanup_work_rates = {"attacking_work_rate":     {"Low": 1, "Medium": 2, "High": 3},
                "defensive_work_rate": {"Low": 1, "Medium": 2, "High": 3}}
FifaBoth_DFs = FifaBoth_DFs.replace(cleanup_work_rates)
new_FifaBoth_DFs = FifaBoth_DFs.loc[:, ~FifaBoth_DFs.columns.isin(['short_name','club_name','player_positions', 'preferred_foot', 'value_eur', 'Values'])]
normalisation = MinMaxScaler() 
array_normalised = normalisation.fit_transform(new_FifaBoth_DFs) 
columns_not_being_normalised = ['short_name','club_name','player_positions', 'preferred_foot', 'value_eur', 'Values']
FifaBoth_DFs_normalised = pd.DataFrame(array_normalised, columns=new_FifaBoth_DFs.columns,index=new_FifaBoth_DFs.index)
joined_dataframe = pd.concat([FifaBoth_DFs[columns_not_being_normalised],FifaBoth_DFs_normalised], axis = 1)
joined_dataframe['player_positions_one'] = joined_dataframe['player_positions'].map(lambda x: x.split(',')[0])
joined_dataframe.drop(columns = ['player_positions'], inplace=True) 
cleanup_player_positions = {"player_positions_one":     {"GK": 1, "LWB": 2, "LB": 3, "CB": 4, "RWB": 5, "RB": 6, "CDM": 7, "CM": 8, "LM": 9,"RM": 10, "CAM": 11, "CF": 12, "LW": 13, "RW": 14, "ST": 15 }}
joined_dataframe = joined_dataframe.replace(cleanup_player_positions)
list_of_positions = joined_dataframe.player_positions_one.unique().tolist()
joined_dataframe.drop(columns = ['club_name', 'league_rank'], inplace=True) 
cleanup_preferred_foot = {"preferred_foot":     {"Left": 0, "Right": 1}}
joined_dataframe = joined_dataframe.replace(cleanup_preferred_foot)

In [ ]:
joined_dataframe

# Data Analysis Section 

In [ ]:
boxplot = joined_dataframe.boxplot(column=['Values']) 

In [ ]:
maxValue = joined_dataframe['Values'].max()

In [ ]:
joined_dataframe[joined_dataframe['Values'] == maxValue]

In [ ]:
#Plot frequency of player value ranges in €5 million intervals
fig = plt.figure(figsize = (10,7))
#Use 20 bins of player values. Value divided by 10^6 so x-axis units are in millions
(joined_dataframe['Values']/10**6).plot.hist(bins = 20)
plt.ticklabel_format(style = 'plain', axis = 'x')
plt.xticks(np.arange(0,125,10))
plt.xlabel('Player values (€M)')
plt.show()

# More Cleaning

In [ ]:

goalkeepers = ['GK']
goalkeepers_dataframe = joined_dataframe[joined_dataframe['player_positions_one'] == 1] 

removing_goalkeeping_stats = ["gk_speed"]
outfield_dataframe = joined_dataframe[joined_dataframe['player_positions_one'] != 1]

outfield_dataframe.drop(columns = removing_goalkeeping_stats, inplace=True) 

deletedRows = ["shooting", "pace","passing", "dribbling", "defending", "physic"]

goalkeepers_dataframe.drop(columns = deletedRows, inplace=True) 
print(outfield_dataframe.columns)

playerNameCol = ["short_name"]

copygoalkeepers_dataframe = goalkeepers_dataframe.copy()
copygoalkeepers_dataframe.drop(columns = playerNameCol, inplace=True) 
copyoutfielddf = outfield_dataframe.copy()
copyoutfielddf.drop(columns = playerNameCol, inplace=True) 

In [ ]:
copyoutfielddf.info()

In [ ]:
copyoutfielddf.drop(['release_clause_eur', 'defending_marking', 'value_eur'], axis = 1, inplace = True) 

In [ ]:
copygoalkeepers_dataframe

In [ ]:
copygoalkeepers_dataframe.info()

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from math import sqrt

def EvaluativeMethods(actualVals, predictedVals):
  array = []
  rmse = sqrt(mean_squared_error(actualVals, predictedVals))
  mse = mean_squared_error(actualVals,predictedVals)
  mae = mean_absolute_error(actualVals,predictedVals)
  rSquared = r2_score(actualVals, predictedVals)

  return rmse, mse, mae, rSquared

In [ ]:
#import required packages
from sklearn import neighbors
from sklearn.metrics import mean_squared_error 
from math import sqrt
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
%matplotlib inline
import catboost as cb
import numpy as np
import pandas as pd
import seaborn as sns
import shap
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.inspection import permutation_importance
import pandas
from sklearn import linear_model
import matplotlib.pyplot as plt

In [ ]:
def plotFeatureImportanceAsGraph(listOfFeatures,listOfFeatureImportanceVals, modelType):
  data={'feature_names':listOfFeatures,'feature_importance':listOfFeatureImportanceVals}
  df = pd.DataFrame(data)

  #Sort the DataFrame in order decreasing feature importance
  df.sort_values(by=['feature_importance'], ascending=False,inplace=True)

  #Define size of bar plot
  plt.figure(figsize=(10,8))
  #Plot Searborn bar chart
  sns.barplot(x=df['feature_importance'], y=df['feature_names'])
  #Add chart labels
  plt.title(modelType + ' FEATURE IMPORTANCE')
  plt.xlabel('FEATURE IMPORTANCE')
  plt.ylabel('FEATURE NAMES')

In [ ]:
'''
FUNCTION: CheckForNaNsInColumn

INPUT: 
dataframe holding all of the information for the football players
column_name = the column that the null values will be checked again

OUTPUT: does not return anything prints the total of null values within a column


'''

def checkForNaNsInColumn(dataframe, column_name):
  print(column_name)
  print(dataframe[column_name].isnull().sum())
  
for column in copygoalkeepers_dataframe.columns:
  checkForNaNsInColumn(copygoalkeepers_dataframe,column)

In [ ]:
for column in copyoutfielddf.columns:
  checkForNaNsInColumn(copyoutfielddf,column)

In [ ]:
copygoalkeepers_dataframe.drop(['release_clause_eur', 'defending_marking', 'value_eur','skill_moves'], axis = 1, inplace = True) 

In [ ]:
copygoalkeepers_dataframe.columns

In [ ]:
# optional: resize images from now on
plt.rcParams["figure.figsize"] = (10, 10)

# numeric_only_columns is a list of columns of the DataFrame
# containing numerical data only
# annot = True to visualize the correlation factor

sns.heatmap(copygoalkeepers_dataframe.corr(), annot = False)
plt.show()

In [ ]:
# optional: resize images from now on
plt.rcParams["figure.figsize"] = (10, 10)

# numeric_only_columns is a list of columns of the DataFrame
# containing numerical data only
# annot = True to visualize the correlation factor

sns.heatmap(copyoutfielddf.corr(), annot = False)
plt.show()

# Run the following models on the transfermarkt data:
**LR**

**KNN**

**Random Forest**

**Decision Tree**

**Linear SVR Model**

**XGBoost**

Linear Regression Goalkeepers


In [ ]:
train , test = train_test_split(copygoalkeepers_dataframe, test_size = 0.2)

x_train = train.drop('Values', axis=1)
y_train = train['Values']


x_test = test.drop('Values', axis = 1)
y_test = test['Values']

regrLRGoalie = linear_model.LinearRegression()
regrLRGoalie.fit(x_train, y_train)

In [ ]:
y_predLRGoalie = regrLRGoalie.predict(x_test)

In [ ]:
EvaluativeMethods(y_test,y_predLRGoalie)

In [ ]:
r2_score(y_test, y_predLRGoalie)

In [ ]:
explainer = shap.LinearExplainer(regrLRGoalie, x_test)
shap_values = explainer.shap_values(x_test)
shap.summary_plot(shap_values, x_test)

In [ ]:
plotFeatureImportanceAsGraph(regrLRGoalie.feature_names_in_, regrLRGoalie.coef_, "Goalkeeping Linear Regression model")

# XGBoostRegressor

In [ ]:
train , test = train_test_split(copygoalkeepers_dataframe,shuffle = True, test_size = 0.2)

x_train = train.drop('Values', axis=1)
y_train = train['Values']


x_test = test.drop('Values', axis = 1)
y_test = test['Values']

xgbr = XGBRegressor(n_estimators=1000,learning_rate=0.1)

In [ ]:
xgbr_model = xgbr.fit(x_train, y_train)

In [ ]:
est = xgbr_model.predict(x_test)

In [ ]:
EvaluativeMethods(y_test,est)

In [ ]:
explainer = shap.TreeExplainer(xgbr_model, x_test)
shap_values = explainer.shap_values(x_test)
shap.summary_plot(shap_values, x_test)

In [ ]:
x_train.columns.tolist()

In [ ]:
featuresXGbrImportance = xgbr_model.feature_importances_

In [ ]:
plotFeatureImportanceAsGraph(x_train.columns.tolist(), featuresXGbrImportance, "Goalkeeper XGBoost model")

Linear Regression Outfield Players


In [ ]:
copyoutfielddf.dropna(subset=['contract_valid_until'],inplace=True)

In [ ]:
copyoutfielddf.columns

In [ ]:
train , test = train_test_split(copyoutfielddf,shuffle = True, test_size = 0.2)

x_train = train.drop('Values', axis=1)
y_train = train['Values']


x_test = test.drop('Values', axis = 1)
y_test = test['Values']

regrLROutfield = linear_model.LinearRegression()
regrLROutfield.fit(x_train, y_train)

In [ ]:
y_predLROutfield = regrLROutfield.predict(x_test)

In [ ]:
EvaluativeMethods(y_test,y_predLROutfield)

X axis is in the outcome variable 

In [ ]:
explainer = shap.LinearExplainer(regrLROutfield, x_test)
shap_values = explainer.shap_values(x_test)
shap.summary_plot(shap_values, x_test)

In [ ]:
plotFeatureImportanceAsGraph(regrLROutfield.feature_names_in_, regrLROutfield.coef_, "Outfield Linear Regression model")

XGRegressor

In [ ]:
train , test = train_test_split(copyoutfielddf, test_size = 0.2)

x_train = train.drop('Values', axis=1)
y_train = train['Values']


x_test = test.drop('Values', axis = 1)
y_test = test['Values']

xgbrOutfield = XGBRegressor(n_estimators=1000,learning_rate=0.1)

In [ ]:
xgbr_modelOutfield = xgbrOutfield.fit(x_train, y_train)

In [ ]:
estOutfield = xgbr_modelOutfield.predict(x_test)

In [ ]:
features_xgbrImportance = xgbr_modelOutfield.feature_importances_

In [ ]:
x_test.columns.tolist()

In [ ]:
EvaluativeMethods(y_test,estOutfield)

In [ ]:
explainer = shap.TreeExplainer(xgbr_modelOutfield, x_test)
shap_values = explainer.shap_values(x_test)
shap.summary_plot(shap_values, x_test)

In [ ]:
plotFeatureImportanceAsGraph(x_test.columns.tolist(), features_xgbrImportance, "Outfield XGBoost model")

KNN Outfield Players 

In [ ]:
copyoutfielddf.columns

In [ ]:
#dataframe with the outcome values

#dataframe with the values not including outcome value


train , test = train_test_split(copyoutfielddf, test_size = 0.2)

x_train = train.drop('Values', axis=1)
y_train = train['Values']

x_test = test.drop('Values', axis = 1)
y_test = test['Values']

knnOutfield = neighbors.KNeighborsRegressor()

# params = {'n_neighbors':[2,3,4,5,6,7,8,9]}

# model = GridSearchCV(knn, params, cv=5)
knnOutfield.fit(x_train,y_train)
#model.best_params_

In [ ]:
y_predKNNOutfield = knnOutfield.predict(x_test)

In [ ]:
EvaluativeMethods(y_test,y_predKNNOutfield)

In [ ]:
x_test.iloc[10]

In [ ]:
explainer = shap.KernelExplainer(knnOutfield.predict, x_test)
shap.initjs()
# Get shap values for the test data observation whose index is 0, i.e. first observation in the test set
shap_values = explainer.shap_values(x_test)

shap.summary_plot(shap_values, x_test)

Goalkeeper dataframe using KNN Model

In [ ]:
#dataframe with the outcome values

#dataframe with the values not including outcome value


train , test = train_test_split(copygoalkeepers_dataframe, test_size = 0.2)

x_train = train.drop('Values', axis=1)
y_train = train['Values']

x_test = test.drop('Values', axis = 1)
y_test = test['Values']

knnGoalie = neighbors.KNeighborsRegressor()

#model = GridSearchCV(knn, params, cv=5)
knnGoalie = knnGoalie.fit(x_train,y_train)
y_predknnGoalie = knnGoalie.predict(x_test)

In [ ]:
EvaluativeMethods(y_test,y_predknnGoalie)

In [ ]:
explainer = shap.KernelExplainer(knnGoalie.predict, x_test)
shap.initjs()
# Get shap values for the test data observation whose index is 0, i.e. first observation in the test set
shap_values = explainer.shap_values(x_test)

shap.summary_plot(shap_values, x_test)

# Random Forest Model

Outfield Players

In [ ]:
train , test = train_test_split(copyoutfielddf, shuffle=True, test_size = 0.2)

x_train = train.drop('Values', axis=1)
y_train = train['Values']


x_test = test.drop('Values', axis = 1)
y_test = test['Values']

regressorRFOutfield = RandomForestRegressor(n_estimators = 20)
regressorRFOutfield.fit(x_train, y_train) 

In [ ]:
x_test.iloc[[12]]

In [ ]:
Y_predregressorRFOutfield = regressorRFOutfield.predict(x_test)

In [ ]:
EvaluativeMethods(y_test,Y_predregressorRFOutfield)

In [ ]:
explainer = shap.TreeExplainer(regressorRFOutfield)
shap.initjs()
shap_values = explainer.shap_values(x_test)
shap.summary_plot(shap_values, x_test)

In [ ]:
features = x_test.columns.tolist()

In [ ]:
random_forest_feature_importance = regressorRFOutfield.feature_importances_

In [ ]:
plotFeatureImportanceAsGraph(features, random_forest_feature_importance, "Outfield Random Forest model")

**Goalkeepers**

In [ ]:
train , test = train_test_split(copygoalkeepers_dataframe,shuffle = True, test_size = 0.2)

x_train = train.drop('Values', axis=1)
y_train = train['Values']


x_test = test.drop('Values', axis = 1)
y_test = test['Values']

regressorRFGoalie = RandomForestRegressor(n_estimators = 100)
regressorRFGoalie.fit(x_train, y_train) 

In [ ]:
Y_predregressorRFGoalie = regressorRFGoalie.predict(x_test)

In [ ]:
EvaluativeMethods(y_test,Y_predregressorRFGoalie)

In [ ]:
explainer = shap.TreeExplainer(regressorRFGoalie)

In [ ]:
x_test.iloc[12]

In [ ]:
explainer = shap.TreeExplainer(regressorRFGoalie)
shap.initjs()
shap_values = explainer.shap_values(x_test)
shap.summary_plot(shap_values, x_test)

In [ ]:
# choosen_instance = x_test.iloc[12]
# shap_values = explainer.shap_values(choosen_instance)
# shap.initjs()
# shap.force_plot(explainer.expected_value, shap_values, choosen_instance)

In [ ]:
features = x_train.columns

In [ ]:
random_forest_feature_importance = regressorRFGoalie.feature_importances_

In [ ]:
random_forest_feature_importance

In [ ]:
plotFeatureImportanceAsGraph(features, random_forest_feature_importance, "Goalkeeper Random Forest model")

# Decision Tree Model

Outfield Players

In [ ]:
from sklearn.tree import DecisionTreeRegressor
train , test = train_test_split(copyoutfielddf, test_size = 0.2)

x_train = train.drop('Values', axis=1)
y_train = train['Values']


x_test = test.drop('Values', axis = 1)
y_test = test['Values']

DTtree_regOutfield = DecisionTreeRegressor()
DTtree_regOutfield.fit(x_train, y_train)
Y_predDTOutfield = DTtree_regOutfield.predict(x_test)

In [ ]:
explainer = shap.TreeExplainer(DTtree_regOutfield)

In [ ]:
x_test.iloc[[13]]

In [ ]:
EvaluativeMethods(y_test,Y_predDTOutfield)

In [ ]:
DTtree_regOutfield.predict(x_test.iloc[[13]])

In [ ]:
explainer = shap.TreeExplainer(DTtree_regOutfield)
shap.initjs()
shap_values = explainer.shap_values(x_test)
shap.summary_plot(shap_values, x_test)

In [ ]:
importance = DTtree_regOutfield.feature_importances_
DTtree_regOutfield.feature_names = (x_train.columns.values)

In [ ]:
plotFeatureImportanceAsGraph(DTtree_regOutfield.feature_names, importance, "Outfield Decision Tree model")

Goalkeepers

In [ ]:
from sklearn.tree import DecisionTreeRegressor
train , test = train_test_split(copygoalkeepers_dataframe, test_size = 0.2)

x_train = train.drop('Values', axis=1)
y_train = train['Values']


x_test = test.drop('Values', axis = 1)
y_test = test['Values']

DTtree_regGoalie = DecisionTreeRegressor()
DTtree_regGoalie.fit(x_train, y_train)
y_predGoalDT = DTtree_regGoalie.predict(x_test)

In [ ]:
EvaluativeMethods(y_test,y_predGoalDT)

In [ ]:
importance = DTtree_regGoalie.feature_importances_
DTtree_regGoalie.feature_names = (x_train.columns.values)

In [ ]:
explainer = shap.TreeExplainer(DTtree_regGoalie)
shap.initjs()
shap_values = explainer.shap_values(x_test)
shap.summary_plot(shap_values, x_test)

In [ ]:
plotFeatureImportanceAsGraph(DTtree_regGoalie.feature_names, importance, "Goalkeeper Decision Tree model")

# Linear SVR

Outfield Players

In [ ]:
from sklearn import svm
train , test = train_test_split(copyoutfielddf, test_size = 0.2)

x_train = train.drop('Values', axis=1)
y_train = train['Values']


x_test = test.drop('Values', axis = 1)
y_test = test['Values']

regrLRSVROutfield = svm.LinearSVR()
regrLRSVROutfield.fit(x_train, y_train)

Y_predLRSVROutfield = regrLRSVROutfield.predict(x_test)

In [ ]:
importance = regrLRSVROutfield.coef_

In [ ]:
EvaluativeMethods(y_test,Y_predLRSVROutfield)

In [ ]:
plotFeatureImportanceAsGraph(regrLRSVROutfield.feature_names_in_, importance, "Outfield Linear SVR model")

In [ ]:
explainer = shap.LinearExplainer(regrLRSVROutfield, x_test)
shap.initjs()
shap_values = explainer.shap_values(x_test)
shap.summary_plot(shap_values, x_test)

Goalkeepers

In [ ]:
from sklearn import svm
train , test = train_test_split(copygoalkeepers_dataframe, test_size = 0.2)

x_train = train.drop('Values', axis=1)
y_train = train['Values']


x_test = test.drop('Values', axis = 1)
y_test = test['Values']

regrLRSVRGoalie = svm.LinearSVR()
regrLRSVRGoalie.fit(x_train, y_train)

Y_predregrLRSVRGoalie = regrLRSVRGoalie.predict(x_test)

In [ ]:
importance = regrLRSVRGoalie.coef_

In [ ]:
EvaluativeMethods(y_test,Y_predregrLRSVRGoalie)

In [ ]:
explainer = shap.LinearExplainer(regrLRSVRGoalie, x_test)
shap.initjs()
shap_values = explainer.shap_values(x_test)
shap.summary_plot(shap_values, x_test)

In [ ]:
plotFeatureImportanceAsGraph(regrLRSVRGoalie.feature_names_in_, importance, "Goalkeeper Linear SVR model")